# Multi Objective Multi Agent Pathfinding Subject to Vehicle Models

## Overview
- loading packages
- Performing single runs
- Visualizing single runs
- Visualising multiple runs from the DB
- Running multiple experiments and saving to DB

## Objectives
- Makespan: Number of steps of the longest path
- Flowtime: Mean number of steps for all agents
- Robustness:
  * Positive: Shortest distance 
    * of an agent to other agents
    * half the distance to the wall
    * reasoning: an agent has radius r and the bigger r could be the better, min distance between two agents is twice agents to wall
  * Negative: In case an agent crosses through an obstacle fraction of the infeasible steps

In [ ]:
%pip install --upgrade pip 
%pip install --upgrade numpy dubins deap matplotlib pandas ipympl seaborn ipywidgets sqlalchemy gitpython nbstripout pre-commit

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, animation
import itertools
from IPython.display import display
import pandas as pd
import seaborn as sns
import cProfile
import pstats

import ipywidgets as widgets

from deap import base, creator, tools, algorithms

rc("animation", html="jshtml")

from path import *
from obstacle_map import *
from problem import *
from experiment import *

import sqlalchemy

engine = sqlalchemy.create_engine(get_key(filename="db.key"))

## Running the Algorithm

In [ ]:
settings = {
    'radius': 10, # turning radius (dubins vehicle)
    'model': Vehicle.DUBINS, # vehicle model
    'step': 1, # step size for simulated behaviour
    'domain': (0, 200.0), # area of operation (-100, 100) means that the vehicles will move in a square from (x=-100, y=-100) to (x=100, y=100)
    'n_agents': 5, # number of agents
    'n_waypoints': 3, # waypoints per agent (excluding start and end)
    'n_gens': 100, # number of generations to run the algorithm
    'population_size': 4*10, # population size for the algorithm, shoulod be divisible by 4 for nsga2
    'cxpb': .5, # crossover probablity
    'mutpb': .5, # mutation rate (not applicable in nsga2)
    'mutation_p': (1.0, 4.0, 5.0), # distribution of mutation types
    'sigma' : 0.2, # sigma for gauss-distribution in waypoint-gauss-mutation
    'feasiblity_threshold': 95, # how robust a solution has to be to be regarded feasible (100-min_dist)
    'offset': (0, 0), # offset of the map to the agents
    'map_name': "cross.obstacles.npy", # name of the obstacle-map file
    'metric': Metric.MIXED, # metric to use in fitness calculation
    'hv_ref': (100, 400), # reference for hyper volume
}

In [ ]:
profiling = False
experiment = Experiment(settings) # load the settings
experiment.setup() # setup population and deap-toolbox
experiment.seed(42)
if profiling:
    profile = cProfile.Profile()
    profile.enable()
pop, logbook = experiment.run(verbose=True) # start running :)
if profiling:
    profile.disable()

In [ ]:
if profiling:
    stats = pstats.Stats(profile)
    stats.sort_stats("tottime")
    stats.print_stats()

## Visualization of single runs

- plot general data
- plot best solutions
- animation for best solution (use filename="FOO.mp4" to save a video file)
- visualize mutation and crossover operators

In [ ]:
# select 5 best individuals (non-dominated sorting)
best = experiment.toolbox.select(pop, 5)

In [ ]:
for ind in best:
    print(ind.fitness.values)
    experiment.problem.solution_plot(ind, plot_range=range(0, 200))

In [ ]:
for i, sol in enumerate(best):
    experiment.problem.solution_animation(sol, plot_range=range(0,200))#, filename=f"with_obstancle_{i}.mp4")

In [ ]:
sol = toolbox.individual()
problem.solution_plot(sol, plot_range=range(0, 200))
print(sol)
print(problem.encode(problem.decode(sol)))

problem.uniform_mutation(sol, debug = True)
print(sol)
problem.solution_plot(sol, plot_range=range(0, 200))
problem.mutate(sol)
problem.solution_plot(sol, plot_range=range(0, 200))
problem.skip_mutation(sol, debug=True)
problem.solution_plot(sol, plot_range=range(0, 200))
#problem.waypoints_to_path(problem.decode(sol))

In [ ]:
plt.close('all')

## Saving and Visualisation - Multiple runs with DB

* works with `sqlalchemy` package and `sqlite` in dev environment
* currently uses `experiments.db` saved to `engine` variable
* adding and removing jobs to the db
* running jobs
* visualisation

In [ ]:
settings = {
    'radius': 10,
    'step': 1,
    'domain': (0, 200.0),
    'n_agents': 5,
    'n_waypoints': 3,
    'n_gens': 500,
    'population_size': 4*25,
    'cxpb': 0.3,
    'mutpb': 1.0,
    'mutation_p': (1.0, 1.0, 1.0),
    'sigma' : 0.2,
    'model': Vehicle.DUBINS,
    'feasiblity_threshold': 95,
    'offset': (0, 0),
    'map_name': "cross.obstacles.npy",
    'metric': Metric.MIN,
    'hv_ref': (100, 400),
}

job_settings = {
    "delete" : True,
    "runs" : 31,
    "experiment" : "dubins_baseline",
    "group" : "default",
    "user" : "basti",
    "db" : engine,
}
add_jobs_to_db(settings, **job_settings)
settings["n_gens"] = 200
job_settings["delete"] = False

s = settings.copy()
j = job_settings.copy()
j["group"] = "dubins_mutation"
for a in np.linspace(0.0, 2.0, num=5):
    for b in np.linspace(0.0, 2.0, num=5):
        for c in np.linspace(0.0, 2.0, num=5):
            if a == 0.0 and b == 0.0 and c == 0.0:
                continue
            s["mutation_p"] = (a, b, c)
            j["experiment"] = f"dubins_mutation_{a:.1f}_{b:.1f}_{c:.1f}"
            add_jobs_to_db(s, **j)
            
s = settings.copy()
s["model"] = Vehicle.STRAIGHT
j = job_settings.copy()
j["group"] = "straight_mutation"
for a in np.linspace(0.0, 2.0, num=5):
    for b in np.linspace(0.0, 2.0, num=5):
        for c in np.linspace(0.0, 2.0, num=5):
            if a == 0.0 and b == 0.0 and c == 0.0:
                continue
            s["mutation_p"] = (a, b, c)
            j["experiment"] = f"straight_mutation_{a:.1f}_{b:.1f}_{c:.1f}"
            add_jobs_to_db(s, **j)
            

s = settings.copy()
j = job_settings.copy()
j["group"] = "dubins_cx"
for i in np.linspace(0.0, 1.0, num=11):
    s["cxpb"] = i
    j["experiment"] = f"dubins_cx_{i:.1f}"
    add_jobs_to_db(s, **j)
    
s = settings.copy()
j = job_settings.copy()
s["model"] = Vehicle.STRAIGHT
j["group"] = "straight_cx"
for i in np.linspace(0.0, 1.0, num=11):
    s["cxpb"] = i
    j["experiment"] = f"straight_cx_{i:.1f}"
    add_jobs_to_db(s, **j)

In [ ]:
df_jobs = pd.read_sql_table("jobs", con=engine)
for status in range(3):
    print(f"status {status}: {len(df_jobs.loc[df_jobs.status == status])}")
print(df_jobs['experiment'].unique())
df_jobs

In [ ]:
runner = ExperimentRunner(engine)
running = True
while running:
    running = runner.fetch_and_execute()
    

In [ ]:
df_pop = pd.read_sql("populations", con=engine)
plt.figure()
sns.scatterplot(data=df_pop, x="robustness", y="flowtime", palette=None, hue="crowding_distance", style="non_dominated", size_order=[True, False], size="non_dominated")
plt.show()

In [ ]:

df_jobs = pd.read_sql_table("jobs", con=engine)
for status in range(3):
    print(f"status {status}: {len(df_jobs.loc[df_jobs.status == status])}")

    


df_pop, df_stats = read_experiment(engine, verbose=True)
plt.close('all')

In [ ]:
plt.figure()
sns.scatterplot()

In [ ]:
df_stats = df_stats.loc[df_stats.generation % 10 == 0]

In [ ]:
sns.relplot(data=df_stats.loc[df_stats["group"]=="dubins_mutation"], x="generation", y="hv", row="mutp_1", col="mutp_2", hue="mutp_0", style="mutp_0", kind="line", ci=90, estimator=np.median, height=2.5, alpha=0.3)

In [ ]:
with plt.xkcd():
    sns.relplot(data=df_pop.loc[df_pop["non_dominated"]], col=None, row=None, x="robustness", y="flowtime", hue="experiment")

In [ ]:
# plot one run makespan-flowtime trade-off with non dominated solutions highlighted
df = df_pop.loc[df_pop["experiment"].isin(['dubins_mutation145', 'dubins_mutation055', 'dubins_mutation111',
       'dubins_mutation151', 'dubins_mutation115', 'dubins_mutation511'])]
plt.figure()
sns.scatterplot(data=df, x="makespan", y="flowtime", hue="experiment", style="non_dominated", size_order=[True, False], size="non_dominated")
plt.show()

In [ ]:
# plot non dominted solutions for all runs
plt.figure()
sns.scatterplot(data=df.loc[df_pop["non_dominated"]].loc[df_pop["run"] <= 10], x="robustness", y="flowtime", hue="experiment", alpha=.5, style="experiment")#, palette="jet")
plt.show()

In [ ]:
plt.figure()
sns.scatterplot(data=df, x="robustness", y="flowtime", hue="crowding_distance", palette="plasma_r", style="non_dominated", size_order=[True, False], size="non_dominated")
plt.show()

In [ ]:
df_non_dom = df_pop.loc[df_pop["experiment"]=="dubins_baseline"].sort_values("crowding_distance", ascending=True)
for i, row in df_non_dom[:5].iterrows():
    display(row)
    plot_indivdual(row, df_jobs=df_jobs, plot=True, animation=True, animation_file=f"{i}.mp4")


In [ ]:
import traceback
traceback.print_last()

In [ ]:
plt.figure()
sns.lineplot(data=df_log, x="generation", y="f_0_median")
plt.show()
plt.figure()
sns.lineplot(data=df_log, x="generation", y="f_1_median")
plt.show()

In [ ]:
list(df_pop["experiment"].unique())

In [ ]:
JobStatus

In [ ]:
for s in JobStatus:
    print(s)

In [ ]:
df = pd.DataFrame({"o1":[1,2,3,4], "o2":[4,3,2,1], "non_dominated": [True, True, True, True]})
df

In [ ]:
def hypervolume2(ref, df, objective_1=None, objective_2=None):
    x_last = ref[0]
    hv = 0
    for i, x in df.loc[df.non_dominated].sort_values(by=[objective_1], ascending=False).iterrows():
        if x[objective_1] > ref[0]:
            continue
        if x[objective_2] > ref[1]:
            continue
        delta_f1 = x_last - x[objective_1]
        delta_f2 = ref[1] - x[objective_2]
        if delta_f1 > 0 and delta_f2 > 0:
            hv += delta_f1 * delta_f2
        x_last = x[objective_1]
    return hv

hypervolume2((1,1), df, objective_1="o1",objective_2="o2")

In [ ]:
hv = hypervolume2((100, 230), df_pop.loc[df_pop["experiment"].eq("dubins_baseline") & df_pop["run"].eq(1)], objective_1="robustness", objective_2="flowtime")
print(hv)
plt.figure()
sns.scatterplot(data=df_pop.loc[df_pop["experiment"].eq("dubins_baseline") & df_pop["run"].eq(1)], x="robustness",y="flowtime", hue="non_dominated")
plt.show()

In [ ]:
sns.lineplot(df_stats, x="generation", y="hv")

In [ ]:
df = df_stats.groupby(["group","experiment", "generation"]).median()
df = df.reset_index(level=["group", "experiment", "generation"])

In [ ]:
sns.relplot(data=df, x="generation", y="hv", col="mutp_1", row="mutp_2", hue="mutp_0", kind="line")

In [ ]:
df = df_pop.loc[df_pop.group == "dubins_mutation"]

df = df.loc[df.mutp_0 == 0.5]
sns.relplot(data=df, x="robustness", y="flowtime", hue="group_front", style="group_front", row="mutp_1", col="mutp_2", height=2.5, alpha=0.2, size="experiment_front", size_order=[True, False])

In [ ]:
plt.close()

In [ ]:
sns.relplot(df_stats.loc[df_stats.group=="dubins_mutation"])

In [ ]:
df = pd.read_sql_query(sql, con=db)